# pandas 高级教程

In [1]:
import pandas as pd
# pass in column names for each CSV
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols,
                    encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols,
                      encoding='latin-1')

# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('ml-100k/u.item', sep='|', names=m_cols, usecols=range(5),
                     encoding='latin-1')

## Inspection

In [2]:
print(movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 5 columns):
movie_id              1682 non-null int64
title                 1682 non-null object
release_date          1681 non-null object
video_release_date    0 non-null float64
imdb_url              1679 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 65.8+ KB
None


In [3]:
print(movies.dtypes)

movie_id                int64
title                  object
release_date           object
video_release_date    float64
imdb_url               object
dtype: object


描述方法会返回所有的 dtype 为数值类型的列的统计量

In [4]:
print(users.describe())

          user_id         age
count  943.000000  943.000000
mean   472.000000   34.051962
std    272.364951   12.192740
min      1.000000    7.000000
25%    236.500000   25.000000
50%    472.000000   31.000000
75%    707.500000   43.000000
max    943.000000   73.000000


In [5]:
print(movies[20:22])

    movie_id                          title release_date  video_release_date  \
20        21  Muppet Treasure Island (1996)  16-Feb-1996                 NaN   
21        22              Braveheart (1995)  16-Feb-1996                 NaN   

                                             imdb_url  
20  http://us.imdb.com/M/title-exact?Muppet%20Trea...  
21  http://us.imdb.com/M/title-exact?Braveheart%20...  


## Boolean Indexing

In [6]:
# users older than 25
print(users[users.age > 25].head(3))
print('\n')

# users aged 40 AND male
print(users[(users.age == 40) & (users.sex == 'M')].head(3))
print('\n')

# users younger than 30 OR female
print(users[(users.sex == 'F') | (users.age < 30)].head(3))

   user_id  age sex occupation zip_code
1        2   53   F      other    94043
4        5   33   F      other    15213
5        6   42   M  executive    98101


     user_id  age sex  occupation zip_code
18        19   40   M   librarian    02138
82        83   40   M       other    44133
115      116   40   M  healthcare    97232


   user_id  age sex  occupation zip_code
0        1   24   M  technician    85711
1        2   53   F       other    94043
2        3   23   M      writer    32067


## 去除多余索引
如果不带 inplace 参数，设置的索引不会改变原来的DataFrame，他返回一个新的！

In [7]:
print(users.set_index('user_id').head())
print('\n')

print(users.head())
print("\n^^^ I didn't actually change the DataFrame. ^^^\n")

with_new_index = users.set_index('user_id')
print(with_new_index.head())
print("\n^^^ set_index actually returns a new DataFrame. ^^^\n")

         age sex  occupation zip_code
user_id                              
1         24   M  technician    85711
2         53   F       other    94043
3         23   M      writer    32067
4         24   M  technician    43537
5         33   F       other    15213


   user_id  age sex  occupation zip_code
0        1   24   M  technician    85711
1        2   53   F       other    94043
2        3   23   M      writer    32067
3        4   24   M  technician    43537
4        5   33   F       other    15213

^^^ I didn't actually change the DataFrame. ^^^

         age sex  occupation zip_code
user_id                              
1         24   M  technician    85711
2         53   F       other    94043
3         23   M      writer    32067
4         24   M  technician    43537
5         33   F       other    15213

^^^ set_index actually returns a new DataFrame. ^^^



设置 inplace=True, 才可以改变原来的DataFrame

In [8]:
users.set_index('user_id', inplace=True)
print(users.head())

         age sex  occupation zip_code
user_id                              
1         24   M  technician    85711
2         53   F       other    94043
3         23   M      writer    32067
4         24   M  technician    43537
5         33   F       other    15213


## 按照索引定位
注意这个索引是 DataFrame 的索引，不是我们定义的 index 索引

In [9]:
print(users.iloc[99])
print('\n')
print(users.iloc[[1, 50, 300]])

age                  36
sex                   M
occupation    executive
zip_code          90254
Name: 100, dtype: object


         age sex occupation zip_code
user_id                             
2         53   F      other    94043
51        28   M   educator    16509
301       24   M    student    55439


## 按照index/label 定位

In [10]:
print(users.loc[100])
# 当只有一行返回的时候，其实返回的是 Series
print('\n')
print(users.loc[[2,51,301]])

age                  36
sex                   M
occupation    executive
zip_code          90254
Name: 100, dtype: object


         age sex occupation zip_code
user_id                             
2         53   F      other    94043
51        28   M   educator    16509
301       24   M    student    55439


再一次重置索引就可以恢复原来的样子

In [11]:
users.reset_index(inplace=True)
print(users.head())

   user_id  age sex  occupation zip_code
0        1   24   M  technician    85711
1        2   53   F       other    94043
2        3   23   M      writer    32067
3        4   24   M  technician    43537
4        5   33   F       other    15213


# SQL-Like Join
pandas.merge, 参数 how (on, left_on, right_on, left_index, right_index) 可以指定使用什么连接！默认是 innner join.
>how : {'left', 'right', 'outer', 'inner'}, default 'inner'

> - left: use only keys from left frame (SQL: left outer join)

> - right: use only keys from right frame (SQL: right outer join)

> - outer: use union of keys from both frames (SQL: full outer join)

> - inner: use intersection of keys from both frames (SQL: inner join)

In [12]:
left_frame = pd.DataFrame({'key': range(5), 'left_value': ['a', 'b', 'c', 'd', 'e']})
right_frame = pd.DataFrame({'key': range(2,7), 'right_value': ['f', 'g','h', 'i', 'j']})
print(left_frame)
print('\n')
print(right_frame)

   key left_value
0    0          a
1    1          b
2    2          c
3    3          d
4    4          e


   key right_value
0    2           f
1    3           g
2    4           h
3    5           i
4    6           j


## Inner Join(default)

In [13]:
print(pd.merge(left_frame, right_frame, on='key', how='inner'))

   key left_value right_value
0    2          c           f
1    3          d           g
2    4          e           h


以上内部连接就相当与 SQL 语句：
```sql
SELECT left_frame.key, left_frame.left_value, right_frame.right_value
FROM left_frame
INNER JOIN right_frame
    ON left_frame.key = right_frame.key;
```
当我们的左右两个表的主键列的列名不一样的时候，我们可以通过 参数 left_on 和 right_on 来指定 join 需要使用的主键列。

In [14]:
#pd.merge(left_frame, right_frame, left_on='left_key', right_on='right_key')

当我们使用的是 表的索引 index 来做join， 那么可以使用 left_index 或者 right_index,还可以混合使用 index 和 column;

In [15]:
print(pd.merge(left_frame, right_frame, left_on='key', right_index=True))

   key  key_x left_value  key_y right_value
0    0      0          a      2           f
1    1      1          b      3           g
2    2      2          c      4           h
3    3      3          d      5           i
4    4      4          e      6           j


## Left Outer Join

In [16]:
print(pd.merge(left_frame, right_frame, on='key', how='left'))

   key left_value right_value
0    0          a         NaN
1    1          b         NaN
2    2          c           f
3    3          d           g
4    4          e           h


以上左外连接的SQL语句：
```sql
SELECT left_frame.key, left_frame.left_value, right_frame.right_value
FROM left_frame
LEFT JOIN right_frame
    ON left_frame.key = right_frame.key;
```

## Right Outer Join

In [17]:
print(pd.merge(left_frame, right_frame, on='key', how='right'))

   key left_value right_value
0    2          c           f
1    3          d           g
2    4          e           h
3    5        NaN           i
4    6        NaN           j


以上右外连接的SQL语句：
```sql
SELECT right_frame.key, left_frame.left_value, right_frame.right_value
FROM left_frame
RIGHT JOIN right_frame
    ON left_frame.key = right_frame.key;
```

## Full Outer Join

In [18]:
print(pd.merge(left_frame, right_frame, on='key', how='outer'))

   key left_value right_value
0    0          a         NaN
1    1          b         NaN
2    2          c           f
3    3          d           g
4    4          e           h
5    5        NaN           i
6    6        NaN           j


以上全外连接的SQL语句：
```sql
SELECT IFNULL(left_frame.key, right_frame.key) key, left_frame.left_value, right_frame.right_value
FROM left_frame
FULL OUTER JOIN right_frame
    ON left_frame.key = right_frame.key;
```

# Combining
我们可以使用 pandas.concat() 函数对 Series 和 DataFrame 进行组合连接操作。

In [19]:
print(pd.concat([left_frame, right_frame]))

   key left_value right_value
0    0          a         NaN
1    1          b         NaN
2    2          c         NaN
3    3          d         NaN
4    4          e         NaN
0    2        NaN           f
1    3        NaN           g
2    4        NaN           h
3    5        NaN           i
4    6        NaN           j


默认情况下，pd.concat() 会把具有相同 column name 的数据沿着 axis=0 的方向（即第一个维度）组合在一起，最终的结果就是行数增加了！

我们也可以指定 axis=1 使其沿着第二维度做连接，那么他就会将相同 index 的列拼接在一起，最终的结果就是列数增加了！

In [20]:
print(pd.concat([left_frame, right_frame], axis=1))

   key left_value  key right_value
0    0          a    2           f
1    1          b    3           g
2    2          c    4           h
3    3          d    5           i
4    4          e    6           j


# Grouping

In [21]:
headers = ['name', 'title', 'department', 'salary']
chicago = pd.read_csv('./ml-100k/city-of-chicago-salaries.csv', 
                      header=0,
                      names=headers,
                      converters={'salary': lambda x: float(x.replace('$', ''))})
print chicago.head()

                    name                     title        department   salary
0        AARON,  ELVIA J          WATER RATE TAKER       WATER MGMNT  85512.0
1      AARON,  JEFFERY M            POLICE OFFICER            POLICE  75372.0
2    AARON,  KIMBERLEI R  CHIEF CONTRACT EXPEDITER  GENERAL SERVICES  80916.0
3    ABAD JR,  VICENTE M         CIVIL ENGINEER IV       WATER MGMNT  99648.0
4  ABBATACOLA,  ROBERT J       ELECTRICAL MECHANIC          AVIATION  89440.0


In [22]:
by_dept = chicago.groupby('department')
by_dept

In [23]:
print(by_dept.count().head()) # NOT NULL records within each column
print('\n')
print(by_dept.size().tail()) # total records for each department
print('\n')
print(by_dept.sum()[20:25]) # total salaries of each department
print('\n')
print(by_dept.mean()[20:25]) # average salary of each department
print('\n')
print(by_dept.median()[20:25]) # take that, RDBMS!

                   name  title  salary
department                            
ADMIN HEARNG         42     42      42
ANIMAL CONTRL        61     61      61
AVIATION           1218   1218    1218
BOARD OF ELECTION   110    110     110
BOARD OF ETHICS       9      9       9


department
PUBLIC LIBRARY     926
STREETS & SAN     2070
TRANSPORTN        1168
TREASURER           25
WATER MGMNT       1857
dtype: int64


                       salary
department                   
HUMAN RESOURCES     4850928.0
INSPECTOR GEN       4035150.0
IPRA                7006128.0
LAW                31883920.2
LICENSE APPL COMM     65436.0


                         salary
department                     
HUMAN RESOURCES    71337.176471
INSPECTOR GEN      80703.000000
IPRA               82425.035294
LAW                70853.156000
LICENSE APPL COMM  65436.000000


                    salary
department                
HUMAN RESOURCES    68496.0
INSPECTOR GEN      76116.0
IPRA               82524.0
LAW        



```sql
SELECT department, COUNT(DISTINCT title)
FROM chicago
GROUP BY department
ORDER BY 2 DESC
LIMIT 5;
```

In [24]:
by_dept.title.nunique().sort_values(ascending=False)[:5]

department
WATER MGMNT    153
TRANSPORTN     150
POLICE         130
AVIATION       125
HEALTH         118
Name: title, dtype: int64

In [25]:
import numpy as np
def ranker(df):
    """Assigns a rank to each employee based on salary, with 1 being the highest paid.
    Assumes the data is DESC sorted."""
    df['dept_rank'] = np.arange(len(df)) + 1
    return df
chicago.sort_values('salary', ascending=False, inplace=True)
chicago = chicago.groupby('department').apply(ranker)
print(chicago[chicago.dept_rank == 3].head(7))

                         name                            title  \
19175  MINIOTIS,  CONSTANTINE  SUPERINTENDENT'S CHIEF OF STAFF   
3733     CALLAHAN,  MICHAEL E         DEPUTY FIRE COMMISSIONER   
25981          SCOTT,  LOIS A          CHIEF FINANCIAL OFFICER   
14098          JONES,  JOHN W                     PSYCHIATRIST   
31996       ZONSIUS,  MICHAEL     MANAGING DEPUTY COMMISSIONER   
17680    MARTINICO,  JOSEPH P           CHIEF LABOR NEGOTIATOR   
29131       TULLY JR,  JOHN F     MANAGING DEPUTY COMMISSIONER   

           department    salary  dept_rank  
19175          POLICE  185004.0          3  
3733             FIRE  178740.0          3  
25981  MAYOR'S OFFICE  169992.0          3  
14098          HEALTH  159668.6          3  
31996        AVIATION  148644.0          3  
17680             LAW  144036.0          3  
29131   STREETS & SAN  142464.0          3  


In [27]:
user_action = pd.DataFrame({"user": [1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3],
                            "item": ['a', 'a', 'a', 'b', 'b', 'c', 'b', 'b', 'd', 'd', 'f', 'f', 'c','c', 'e', 'e'],
                     "type": [1, 1, 4, 3, 5, 6, 1, 1, 3, 4, 5, 6, 4, 2, 3, 3],
                     "date": ['1', '2', '3', '3', '4', '4', '5', '6','1', '2', '3', '3', '4', '4', '5', '6']},
                     columns=['user','item','type', 'date'])

result = user_action.sort_values(['date'],axis=0).drop_duplicates(subset=['user', 'type'],keep='first')
print result
result.set_index(['user','type'], inplace=True)
print result
print user_action

    user item  type date
0      1    a     1    1
8      2    d     3    1
9      2    d     4    2
2      1    a     4    3
3      1    b     3    3
10     2    f     5    3
11     2    f     6    3
4      1    b     5    4
5      1    c     6    4
12     3    c     4    4
13     3    c     2    4
6      2    b     1    5
14     3    e     3    5
          item date
user type          
1    1       a    1
2    3       d    1
     4       d    2
1    4       a    3
     3       b    3
2    5       f    3
     6       f    3
1    5       b    4
     6       c    4
3    4       c    4
     2       c    4
2    1       b    5
3    3       e    5
    user item  type date
0      1    a     1    1
1      1    a     1    2
2      1    a     4    3
3      1    b     3    3
4      1    b     5    4
5      1    c     6    4
6      2    b     1    5
7      2    b     1    6
8      2    d     3    1
9      2    d     4    2
10     2    f     5    3
11     2    f     6    3
12     3    c     4    4


In [29]:
from collections import Counter
def add_type_count(group):
    user_behaviors = group.type.astype(int)
    type_cnt = Counter(user_behaviors)
    # 1: 浏览 2: 加购 3: 删除
    # 4: 购买 5: 收藏 6: 点击
    group['browse_num'] = type_cnt[1]
    group['addcart_num'] = type_cnt[2]
    group['delcart_num'] = type_cnt[3]
    group['buy_num'] = type_cnt[4]
    group['favor_num'] = type_cnt[5]
    group['click_num'] = type_cnt[6]

    return group[['user', 'item', 'browse_num', 'addcart_num',
                  'delcart_num', 'buy_num', 'favor_num',
                  'click_num']]

# df_ac = user_action.groupby(['user', 'item'], as_index=True).apply(add_type_count)
# print df_ac
# print('\n')
df_ac = user_action.groupby(['user', 'item'], as_index=False).apply(add_type_count)
print df_ac

    user item  browse_num  addcart_num  delcart_num  buy_num  favor_num  \
0      1    a           2            0            0        1          0   
1      1    a           2            0            0        1          0   
2      1    a           2            0            0        1          0   
3      1    b           0            0            1        0          1   
4      1    b           0            0            1        0          1   
5      1    c           0            0            0        0          0   
6      2    b           2            0            0        0          0   
7      2    b           2            0            0        0          0   
8      2    d           0            0            1        1          0   
9      2    d           0            0            1        1          0   
10     2    f           0            0            0        0          1   
11     2    f           0            0            0        0          1   
12     3    c           0

In [31]:
df_ac = user_action.groupby(['user', 'item'], as_index=False).apply(add_type_count).drop_duplicates(subset=['user', 'item'])
print df_ac

    user item  browse_num  addcart_num  delcart_num  buy_num  favor_num  \
0      1    a           2            0            0        1          0   
3      1    b           0            0            1        0          1   
5      1    c           0            0            0        0          0   
6      2    b           2            0            0        0          0   
8      2    d           0            0            1        1          0   
10     2    f           0            0            0        0          1   
12     3    c           0            1            0        1          0   
14     3    e           0            0            2        0          0   

    click_num  
0           0  
3           0  
5           1  
6           0  
8           0  
10          1  
12          0  
14          0  
